In [21]:
# Import libraries
import logging
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import logging
from itertools import chain
import json

In [2]:
import threading


class MultiThreading(object):

    def __init__(self, scrapers):
        self.scrapers = scrapers

    def run(self):
        threads = []

        for i in range(len(self.scrapers)):
            t = threading.Thread(target=self.scrapers[i].start)
            t.start()
            threads.append(t)

        for thread in threads:
            thread.join()


In [3]:
class Scraper(object):
    """
    Pull player info down from web
    """

    # Initialize array to store player
    players_scraped = []

    # Instantiate scraper
    def __init__(self, urls):
        self.urls = urls
        self.logger = logging.getLogger("sLogger")

    # request to get the url
    def get_page(self, url):
        response = requests.get(url)
        if response.status_code:
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.find("tbody", {"class": "list"})
        else:
            self.logger.error("Error" + response.status_code)
            return None

    # helper method to get players
    def get_players(self,trs):
        return [extract_info(tr) for tr in trs]
    
    # helper method to get stats
    def get_stats(self,trs):
        return [extract_stats(tr) for tr in trs]

    # method to extract and copy player info from web
    def scrap(self, urls):
        for url in urls:
            tbody = self.get_page(url)
            if tbody is None:
                continue
            trs = tbody.findAll("tr")
            Scraper.players_scraped.append(self.get_players(trs))
            self.logger.info("Page{} scraped".format(len(Scraper.players_scraped)))
    # method to start the scraper
    def start(self):
        self.scrap(self.urls)

In [5]:

url = "https://sofifa.com/players?offset="
urls = [url + str(offset) for offset in range(0,120, 60)]
# Parameters
number_of_scraper = 5
pages = 10

scrapers = [Scraper(urls[pages * i:min(pages * (i + 1), len(urls))]) 
            for i in range(number_of_scraper)]
multi_threading = MultiThreading(scrapers)

#     multi_threading.run()

In [59]:
# Implement this into our saving csv (more generic name save data perhaps)
# so that it handles saving files to csvs and jsons
# eg two types of json (flat one, with no sub category of attributes,
# and allow sub category: P1 = {"name" : "", ... , "stats":
# {"att": {},  "def": {}, ...}}

with open("../data/raw/fifa.json", "w") as file:
    file.write(json.dumps(dicts, indent=4))


In [51]:
d = {"a": "1",
     "b": "2",
     "c": "3"}
json_obj = json.dumps(d, indent=4)
#print(json_obj)

test_json = json.dumps(dicts, indent = 4)
#print(test_json)
one_level = json.dumps(single_list, indent = 4)
#print(one_level)

In [6]:
def flatten(d):
    out = {}
    for key, val in d.items():
        if isinstance(val, dict):
            val = [val]
        if isinstance(val, list):
            for subdict in val:
                deeper = flatten(subdict).items()
                out.update({key2: val2 for key2, val2 in deeper})
        else:
            out[key] = val
    return out

In [54]:
dicts

[{'name': 'Gianluca Scamacca',
  'country': 'Italy',
  'age': '22',
  'overall': '78',
  'potential': '84',
  'club': 'U.S. Sassuolo Calcio',
  'best_position': 'ST',
  'value': '€23M',
  'wage': '€34K',
  'crossing': '22',
  'Finishing': '80',
  'Heading Accuracy': '78',
  'Short passing': '76',
  'Volleys': '79',
  'Dribbling': '73',
  'Curve': '69',
  'Fk Accuracy': '68',
  'Long Passing': '59',
  'Ball Control': '81',
  'Acceleration': '70',
  'Sprint Speed': '75',
  'Agility': '63',
  'Reactions': '78',
  'Balance': '68',
  'Shot Power': '80',
  'Jumping': '83',
  'Stamina': '67',
  'Strength': '85',
  'Long Shots': '74',
  'Aggression': '55',
  'Interceptions': '26',
  'Positioning': '7',
  'Vision': '59',
  'Penalties': '68',
  'Composure': '78',
  'Defensive Awareness': '40',
  'Standing Tackle': '28',
  'Sliding Tackle': '20',
  'Diving': '12',
  'Handling': '10',
  'Kicking': '11',
  'Reflexes': '12'},
 {'name': 'Lionel Messi',
  'country': 'Argentina',
  'age': '34',
  'over

In [44]:
#single_list = list(chain.from_iterable(array_players))
dicts = list(map(flatten, (chain.from_iterable(array_players))))

In [12]:
t1 = time.time()
multi_threading.run()

array_players = Scraper.players_scraped
players = list(map(flatten, (chain.from_iterable(Scraper.players_scraped))))

df_multi_thread = pd.DataFrame(players)
df_multi_thread.drop_duplicates(ignore_index = True)
print("Multi threading time taken: ", time.time() - t1)
df_multi_thread.head()


Multi threading time taken:  47.564934492111206


,name,country,age,overall,potential,club,best_position,value,wage,crossing,...,Vision,Penalties,Composure,Defensive Awareness,Standing Tackle,Sliding Tackle,Diving,Handling,Kicking,Reflexes
0,Gianluca Scamacca,Italy,22,78,84,U.S. Sassuolo Calcio,ST,€23M,€34K,22,...,59,68,78,40,28,20,12,10,11,12
1,Lionel Messi,Argentina,34,92,92,Paris Saint-Germain,RW,€69.5M,€290K,85,...,95,75,96,20,35,24,6,11,15,8
2,Mykhailo Mudryk,Ukraine,20,73,84,Shakhtar Donetsk,LM,€6.5M,€650,69,...,72,52,70,51,52,38,10,12,14,7
3,Hugo Ekitike,France,19,72,82,Stade de Reims,ST,€5M,€12K,45,...,59,63,73,24,17,16,8,8,7,12
4,Julián Álvarez,Argentina,21,78,86,River Plate,ST,€32M,€87K,75,...,81,73,82,60,53,46,6,10,8,6


In [11]:
# helper method for extracting data of the player

def extract_info(tr):
    base = "https://sofifa.com/"
    link = base + tr.select('td.col-name')[0].find("a").get("href")
    return {
        "name": tr.select('td.col-name')[0].find("a").get("aria-label"),
        "country": tr.select('td.col-name')[0].find("img").get("title"),
        "age": tr.select('td.col.col-ae')[0].text.strip(),
        "overall": tr.select('td.col.col-oa')[0].text.strip(),
        "potential": tr.select('td.col.col-pt')[0].text.strip(),
        "club": tr.select("td.col-name")[1].find("a").text,
        "best_position": tr.select('td.col-name')[0].find("span").text,
        "value": tr.select('td.col.col-vl')[0].text.strip(),
        "wage": tr.select('td.col.col-wg')[0].text.strip(),
        "stats": extract_stats(link)
    }


# helper method for extracting stats of single player

def extract_stats(link):
    new_res = requests.get(link)
    new_soup = BeautifulSoup(new_res.content, "html.parser")
    new_tbody = new_soup.find_all("div", {"class": "center"})[5]
    stats_block = new_tbody.findAll("div", {"class": "block-quarter"})
    return extract_deep(stats_block)


def extract_deep(stats_block):
    return {"Att": extract_att(stats_block[0]),
            "Skill": extract_skill(stats_block[1]), 
            "Move": extract_move(stats_block[2]),
            "Power": extract_pow(stats_block[3]),
            "Mentality": extract_mentality(stats_block[4]),
            "Defending": extract_def(stats_block[5]),
            "Goalkeep": extract_goalkeep(stats_block[6])
            }

In [10]:
def extract_att(att):
    return {"crossing": att.findAll("li")[0].find("span").text.strip(),
            "Finishing": att.findAll("li")[1].find("span").text.strip(),
            "Heading Accuracy":  att.findAll("li")[2].find("span").text.strip(),
            "Short passing": att.findAll("li")[3].find("span").text.strip(),
            "Volleys": att.findAll("li")[4].find("span").text.strip()
            }


def extract_skill(ski):
    return {"Dribbling": ski.findAll("li")[0].find("span").text.strip(),
            "Curve": ski.findAll("li")[1].find("span").text.strip(),
            "Fk Accuracy": ski.findAll("li")[2].find("span").text.strip(),
            "Long Passing": ski.findAll("li")[3].find("span").text.strip(),
            "Ball Control": ski.findAll("li")[4].find("span").text.strip()
            }


def extract_move(mov):
    return {"Acceleration": mov.findAll("li")[0].find("span").text.strip(),
            "Sprint Speed": mov.findAll("li")[1].find("span").text.strip(),
            "Agility": mov.findAll("li")[2].find("span").text.strip(),
            "Reactions": mov.findAll("li")[3].find("span").text.strip(),
            "Balance": mov.findAll("li")[4].find("span").text.strip()
            }


def extract_pow(pow):
    return {"Shot Power": pow.findAll("li")[0].find("span").text.strip(),
            "Jumping": pow.findAll("li")[1].find("span").text.strip(),
            "Stamina": pow.findAll("li")[2].find("span").text.strip(),
            "Strength": pow.findAll("li")[3].find("span").text.strip(),
            "Long Shots": pow.findAll("li")[4].find("span").text.strip()
            }


def extract_mentality(men):
    return {"Aggression": men.findAll("li")[0].find("span").text.strip(),
            "Interceptions": men.findAll("li")[1].find("span").text.strip(),
            "Positioning": men.findAll("li")[2].find("span").text.strip(),
            "Vision": men.findAll("li")[3].find("span").text.strip(),
            "Penalties": men.findAll("li")[4].find("span").text.strip(),
            "Composure": men.findAll("li")[5].find("span").text.strip()
            }


def extract_def(defe):
    return {"Defensive Awareness": defe.findAll("li")[0].find("span").text.strip(),
            "Standing Tackle": defe.findAll("li")[1].find("span").text.strip(),
            "Sliding Tackle": defe.findAll("li")[2].find("span").text.strip()
            }


def extract_goalkeep(gk):
    return {"Diving": gk.findAll("li")[0].find("span").text.strip(),
            "Handling": gk.findAll("li")[1].find("span").text.strip(),
            "Kicking": gk.findAll("li")[2].find("span").text.strip(),
            "Positioning": gk.findAll("li")[3].find("span").text.strip(),
            "Reflexes": gk.findAll("li")[4].find("span").text.strip()
            }



## Prototype

- Refactor the Scraper class, so that it has different branch of statements to handle either scrap surface only or scrap deep only

In [ ]:
pls = []
url = "https://sofifa.com/players?offset="
urls = [url + str(offset) for offset in range(0, 30, 10)]
for url in urls:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    tbody = soup.find("tbody", {"class": "list"})
    trs = tbody.findAll("tr")
    pls.append([extract_info(tr) for tr in trs])

####

first_url = pls[0][0]["link"]
first_url
r = requests.get(first_url)
sou = BeautifulSoup(r.content, "html.parser")
tb = soup.find_all("div", {"class": "center"})[5]
blo = tb.findAll("div", {"class" : "block-quarter"})
type(blo)

###

url = "https://sofifa.com/player/27488/jan-vennegoor-of-hess/120002/"
new_res = requests.get(url)
soup = BeautifulSoup(new_res.content, "html.parser")
tbody = soup.find_all("div", {"class": "center"})[5]
stats_block = tbody.findAll("div", {"class": "block-quarter"})

